# Using nba-dataloader

This notebook takes you through a few examples of downloading data from http://stats.nba.com and using it to do some simple data analysis.

## Installation
Make sure you have installed the nba-dataloader package

In [ ]:
%pip install nba-dataloader

### Make sure package is installed correctly
However before we do that let's check out how to use the module using the --help option

In [ ]:
%run -m nba_dataloader -h

## Downloading data
Lets download some data, for this exercise we will be querying the following resources 
- [LeagueDashPlayerStats](https://any-api.com/nba_com/nba_com/docs/_leaguedashplayerstats/GET)
- [DraftHistory](https://any-api.com/nba_com/nba_com/docs/_drafthistory/GET)

You can see the nba_dataloader package requires one parameter ```resource```. Examining [LeagueDashPlayerStats](https://any-api.com/nba_com/nba_com/docs/_leaguedashplayerstats/GET) resource we see that the resource takes a few required request parameters. 

### Specifying parameters
There are a few ways to specify the request parameters:-
#### Using --params
You can provide a python module containing a variable params of type list of dicts.
#### Using Defaults
If no parameters are provided, the default behavior of the script is to look for a module called ```request_params.<resource>_params```. 
    
In this case if we used the default, then the script will try to load the request parameters from ```request_params.leaguedashplayerstats_params```

If you started the jupyter server in the git repo that you cloned, you will find the module under ```request_params/leaguedashplayerstats.py``` Have a look at the file.

In [ ]:
default_params = {
        "LastNGames": 0,
        "LeagueID": "00",
        "MeasureType": "Base",
        "Month": 0,
        "OpponentTeamID": 0,
        "PORound": 0,
        "PaceAdjust": "N",
        "PerMode": "Totals",
        "Period": 0,
        "PlusMinus": "N",
        "Rank": "N",
        "SeasonType": "Regular Season",
        "TeamID": 0
}
seasons = {'1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05',
           '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14',
           '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23'}
params = map(lambda season: {'Season': season} | default_params, seasons)

The ```map(...)``` operation generates a list of dicts generated by appending the key/value "Season":<season> to the default_params for each of the 27 seasons from 1996-97 to 2022-23. The result can bee seen by running the code below

In [ ]:
list(params)

### Download LeagueDashPlayerstats
Let's run the package using the defaults

In [ ]:
%run -m nba_dataloader leaguedashplayerstats

What just happened??? 

The script queried the endpoint http://stats.nba.com/leaguedashplayerstats 27 times, once each for every dict value in the ```params``` list. Player stats for every season from 1996-97 to 2022-23 was fetched and the results are stored in ```tmp/LeagueDashPlayerStats``` as a delta table. 

### Examine delta table
Let's examine the contents of the delta table using the delta-rs package

In [ ]:
from deltalake import DeltaTable
import pandas as pd

pd.set_option('display.max_columns', None)

dt = DeltaTable("tmp/leaguedashplayerstats")
display(dt.to_pandas())

You will see at total of 12846 players logged minutes from 1996-97 to 2022-23.

## Querying using Spark
Let's see how many players played for each team during this period. This time we will use spark to query the delta tables. 

### Installing delta-spark
However before that we need to install a few more python packages:- pyspark and delta-spark. Ensure you have the compatible versions of the two from [here](https://docs.delta.io/latest/releases.html). We will be installing 
 - [delta-spark==2.4.0](https://pypi.org/project/delta-spark/2.4.0/)
 - [pyspark==3.4.1](https://pypi.org/project/pyspark/3.4.1/)
 
 Installing delta-spark should also install the correct version of pyspark

In [ ]:
%pip install delta-spark==2.4.0

### List number of players who have played for each team

In [ ]:
import os
import sys
import pyspark
from delta.pip_utils import configure_spark_with_delta_pip
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    
builder = pyspark.sql.SparkSession.builder.appName("NBA Analytics")\
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

df = spark.read.format("delta").load("tmp/LeagueDashPlayerStats")
df.createOrReplaceTempView("PLAYER_STATS")
spark.sql("SELECT TEAM_ABBREVIATION, count(TEAM_ABBREVIATION) as NUM_PLAYERS from PLAYER_STATS"\
          " group by TEAM_ABBREVIATION order by NUM_PLAYERS").show(50)

### Download DraftHistory
Let us query another resource: [DraftHistory](https://any-api.com/nba_com/nba_com/docs/_drafthistory/GET). This time we will use a custom python module to pass the parameters.

Run the cell below to create a python file ```drafthistory_params.py``` with the following content

In [ ]:
%%writefile drafthistory_params.py

params = [{
    "LeagueID":"00"
}]

Now run the package to fetch the draft history data

In [ ]:
%run -m nba_dataloader drafthistory --params drafthistory_params

You can see a total of 8257 players have been drafted into the NBA as of the 2023-24 season. 

### List of all the number #1 draft picks.

In [ ]:
drafthistory = spark.read.format("delta").load("tmp/DraftHistory")
drafthistory.createOrReplaceTempView("DRAFT_HISTORY")
spark.sql("SELECT PLAYER_NAME, SEASON, ROUND_NUMBER, ROUND_PICK from DRAFT_HISTORY"\
          " where ROUND_NUMBER=1 and ROUND_PICK=1").show(75)

### All Players drafted by the San Antonio Spurs

In [ ]:
df = spark.sql("SELECT * from DRAFT_HISTORY where TEAM_ID=1610612759")
df.show(n=df.count(), truncate=False)

### Spurs players count by draft organization

In [ ]:
df = spark.sql("WITH SPURS_PICS as ( "\
         "SELECT * from DRAFT_HISTORY where TEAM_ID=1610612759)"
         " SELECT ORGANIZATION, COUNT(ORGANIZATION) as org_count FROM SPURS_PICS GROUP BY ORGANIZATION order by org_count desc")
df.show(n=df.count())

### Using Spark-SQL
You can also use spark-sql command line tool to query that tables. If you have spark installed you would use the following command.
```
spark-sql --packages io.delta:delta-core_2.12:2.4.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"
```

Remember though that while running queries, you should provide the full table path and not relative while referring to the tables.